In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import  ProfileReport
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv(r"C:\Users\USER\Documents\Gomycode_Datasciencecalebclass\Electric_cars_dataset.csv")

In [3]:
df.head(5)

,ID,VIN (1-10),County,City,State,ZIP Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,98037.0,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,32.0,109821694,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,98229.0,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,0,40.0,137375528,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,98422.0,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,0,27.0,150627382,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,98053.0,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,0,45.0,258766301,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,98375.0,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,25.0,296998138,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64353 entries, 0 to 64352
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ID                                                 64353 non-null  object 
 1   VIN (1-10)                                         64353 non-null  object 
 2   County                                             64349 non-null  object 
 3   City                                               64344 non-null  object 
 4   State                                              64342 non-null  object 
 5   ZIP Code                                           64347 non-null  float64
 6   Model Year                                         64346 non-null  float64
 7   Make                                               64349 non-null  object 
 8   Model                                              64340 non-null  object 
 9   Electr

In [5]:
df.describe()

,ZIP Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID
count,64347.000000,64346.000000,64353.000000,64353.000000,64184.000000,6.435300e+04
mean,98143.452888,2018.186212,106.948985,2524.990754,29.951904,1.972905e+08
std,2856.064329,2.726742,104.093919,12402.895104,14.661124,1.069466e+08
min,745.000000,1993.000000,0.000000,0.000000,0.000000,4.385000e+03
25%,98052.000000,2017.000000,14.000000,0.000000,19.000000,1.372865e+08
50%,98121.000000,2018.000000,73.000000,0.000000,34.000000,1.753776e+08
75%,98370.000000,2021.000000,215.000000,0.000000,43.000000,2.299039e+08
max,99701.000000,2022.000000,337.000000,845000.000000,49.000000,4.789346e+08


In [6]:
df.shape

(64353, 18)

In [7]:
df.select_dtypes( include = np.number )

,ZIP Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID
0,98037.0,2020.0,308,0,32.0,109821694
1,98229.0,2011.0,73,0,40.0,137375528
2,98422.0,2015.0,81,0,27.0,150627382
3,98053.0,2013.0,38,0,45.0,258766301
4,98375.0,2019.0,238,0,25.0,296998138
...,...,...,...,...,...,...
64348,98144.0,2020.0,239,0,37.0,156575107
64349,98402.0,2013.0,6,0,27.0,211048701
64350,98119.0,2018.0,238,0,36.0,135543411
64351,98115.0,2017.0,14,0,46.0,192459907


In [8]:
df.select_dtypes(include = object)

,ID,VIN (1-10),County,City,State,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Vehicle Location,Electric Utility,Expected Price ($1k)
0,EV33174,5YJ3E1EC6L,Snohomish,LYNNWOOD,WA,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.287614 47.83874),PUGET SOUND ENERGY INC,50
1,EV40247,JN1AZ0CP8B,Skagit,BELLINGHAM,WA,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.414936 48.709388),PUGET SOUND ENERGY INC,15
2,EV12248,WBY1Z2C56F,Pierce,TACOMA,WA,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.396286 47.293138),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,EV55713,1G1RD6E44D,King,REDMOND,WA,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.024951 47.670286),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,EV28799,1G1FY6S05K,Pierce,PUYALLUP,WA,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.321062 47.103797),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78
...,...,...,...,...,...,...,...,...,...,...,...,...
64348,EV6357,KNDCE3LG7L,King,SEATTLE,WA,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.30033 47.585339),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),33
64349,EV423,JTDKN3DP2D,Pierce,TACOMA,WA,TOYOTA,PRIUS PLUG-IN,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,POINT (-122.443211 47.252172),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,13.3
64350,EV27852,1G1FX6S05J,King,SEATTLE,WA,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,POINT (-122.367721 47.639264),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22.857
64351,EV830,WP1AE2A24H,King,SEATTLE,WA,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,POINT (-122.297534 47.685291),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),45.7


In [9]:
df.nunique()

ID                                                   64353
VIN (1-10)                                            5644
County                                                 139
City                                                   544
State                                                   38
ZIP Code                                               678
Model Year                                              19
Make                                                    34
Model                                                  107
Electric Vehicle Type                                    2
Clean Alternative Fuel Vehicle (CAFV) Eligibility        3
Electric Range                                          98
Base MSRP                                               37
Legislative District                                    50
DOL Vehicle ID                                       64353
Vehicle Location                                       668
Electric Utility                                        

In [10]:
df = df.drop(columns = ['ID' ,'VIN (1-10)', 'ZIP Code', 'Base MSRP', 'DOL Vehicle ID', 'ZIP Code','Legislative District','Vehicle Location'])
df

,County,City,State,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Electric Utility,Expected Price ($1k)
0,Snohomish,LYNNWOOD,WA,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,PUGET SOUND ENERGY INC,50
1,Skagit,BELLINGHAM,WA,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,PUGET SOUND ENERGY INC,15
2,Pierce,TACOMA,WA,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,King,REDMOND,WA,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,Pierce,PUYALLUP,WA,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78
...,...,...,...,...,...,...,...,...,...,...,...
64348,King,SEATTLE,WA,2020.0,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,239,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),33
64349,Pierce,TACOMA,WA,2013.0,TOYOTA,PRIUS PLUG-IN,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,6,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,13.3
64350,King,SEATTLE,WA,2018.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22.857
64351,King,SEATTLE,WA,2017.0,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),45.7


In [11]:
df.shape

(64353, 11)

In [12]:
df = df.rename(columns = {'Expected Price ($1k)':'Expected Price'})
df

,County,City,State,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Electric Utility,Expected Price
0,Snohomish,LYNNWOOD,WA,2020.0,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,PUGET SOUND ENERGY INC,50
1,Skagit,BELLINGHAM,WA,2011.0,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,73,PUGET SOUND ENERGY INC,15
2,Pierce,TACOMA,WA,2015.0,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,18
3,King,REDMOND,WA,2013.0,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,38,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),33.9
4,Pierce,PUYALLUP,WA,2019.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,41.78
...,...,...,...,...,...,...,...,...,...,...,...
64348,King,SEATTLE,WA,2020.0,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,239,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),33
64349,Pierce,TACOMA,WA,2013.0,TOYOTA,PRIUS PLUG-IN,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,6,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,13.3
64350,King,SEATTLE,WA,2018.0,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),22.857
64351,King,SEATTLE,WA,2017.0,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),45.7


In [13]:
df["Expected Price"] = pd.to_numeric(df["Expected Price"],errors = 'coerce')

In [14]:
df["Expected Price"].dtypes

dtype('float64')

In [15]:
pip install ydata_profiling

In [16]:

profiling = ProfileReport(df, title ="data profiling for electric cars")
profiling

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
df.isna().sum()

County                                                 4
City                                                   9
State                                                 11
Model Year                                             7
Make                                                   4
Model                                                 13
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Electric Utility                                     722
Expected Price                                        13
dtype: int64

In [18]:
Numeric = df.select_dtypes(include = np.number).columns
df[Numeric] = df[Numeric].fillna(df[Numeric].mean())
#method 2
#Numeric = df.select_dtypes(include = np.number)
#df[Numeric.columns] = df[Numeric.columns].fillna(df[Numeric.columns].mean())


In [19]:
for col in df.select_dtypes( include = object):
    df[col] = df[col].fillna(df[col].mode()[0])

In [20]:
df.isna().sum()

County                                               0
City                                                 0
State                                                0
Model Year                                           0
Make                                                 0
Model                                                0
Electric Vehicle Type                                0
Clean Alternative Fuel Vehicle (CAFV) Eligibility    0
Electric Range                                       0
Electric Utility                                     0
Expected Price                                       0
dtype: int64

In [21]:
df.duplicated().sum()
df = df.drop_duplicates()
df.duplicated().sum()

0

In [22]:
Numbers = df.select_dtypes(include = np.number)
zscoress = zscore(Numbers)

In [23]:
zscoress.nlargest(2,Numbers)

,Model Year,Electric Range,Expected Price
1894,1.459579,-0.503153,0.624
3098,1.459579,-0.503153,0.624


In [24]:
Numbers =Numbers[np.abs(zscoress) < 2.5]


In [25]:
Numbers.fillna(Numbers.mean(), inplace = True)

In [26]:
df.loc[:, ['Model Year','Electric Range','Expected Price']] = Numbers

C:\Users\USER\AppData\Local\Temp\ipykernel_16880\1182051903.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[308.  73.  81. ...  21.  32. 150.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, ['Model Year','Electric Range','Expected Price']] = Numbers


In [27]:
df.isna().sum()

County                                               0
City                                                 0
State                                                0
Model Year                                           0
Make                                                 0
Model                                                0
Electric Vehicle Type                                0
Clean Alternative Fuel Vehicle (CAFV) Eligibility    0
Electric Range                                       0
Electric Utility                                     0
Expected Price                                       0
dtype: int64

In [28]:
df.dtypes

County                                                object
City                                                  object
State                                                 object
Model Year                                           float64
Make                                                  object
Model                                                 object
Electric Vehicle Type                                 object
Clean Alternative Fuel Vehicle (CAFV) Eligibility     object
Electric Range                                       float64
Electric Utility                                      object
Expected Price                                       float64
dtype: object

In [29]:
labelencoder = LabelEncoder()

df['City'] = labelencoder.fit_transform(df['City'])
df['County'] = labelencoder.fit_transform(df['County'])
df['State'] = labelencoder.fit_transform(df['State'])
df['Make'] = labelencoder.fit_transform(df['Make'])
df['Model'] = labelencoder.fit_transform(df['Model'])
df['Electric Vehicle Type'] = labelencoder.fit_transform(df['Electric Vehicle Type'])
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = labelencoder.fit_transform(df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'])
df['Electric Utility'] = labelencoder.fit_transform(df['Electric Utility'])

C:\Users\USER\AppData\Local\Temp\ipykernel_16880\1549812319.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = labelencoder.fit_transform(df['City'])
C:\Users\USER\AppData\Local\Temp\ipykernel_16880\1549812319.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['County'] = labelencoder.fit_transform(df['County'])
C:\Users\USER\AppData\Local\Temp\ipykernel_16880\1549812319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [30]:
x = df.drop('Expected Price', axis=1)
y = df['Expected Price']

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25, random_state = 42)

In [32]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

In [33]:
svm = SVR(kernel = 'linear')
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

In [34]:
data = pd.DataFrame({"real_values": y_test,"Predicted_lables": y_pred})
data.head()

,real_values,Predicted_lables
8438,37.68543,42.466253
36865,54.00000,42.176706
17255,65.00000,38.199359
42097,18.00000,18.100044
9202,82.70000,47.808089


In [35]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R²) Score:", r2)
#note mean_square_error use be lower than the square means of the target column

Mean Absolute Error (MAE): 11.948404238408953
Mean Squared Error (MSE): 272.72971124645227
R-squared (R²) Score: 0.4194682055826072


In [36]:
df.describe()

,County,City,State,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Electric Utility,Expected Price
count,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000
mean,78.735266,278.532776,34.813207,2017.845046,17.206218,55.223839,0.413339,0.682884,90.201409,51.242904,37.685430
std,34.803460,159.214125,2.156696,2.810676,10.216428,24.861250,0.492447,0.861171,96.064364,19.865458,21.661063
min,0.000000,0.000000,0.000000,2011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60.000000,145.000000,35.000000,2016.000000,6.000000,40.000000,0.000000,0.000000,18.000000,32.000000,20.000000
50%,62.000000,280.000000,35.000000,2018.000000,19.000000,57.000000,0.000000,0.000000,40.000000,65.000000,31.670000
75%,120.000000,433.000000,35.000000,2020.000000,28.000000,67.000000,1.000000,2.000000,200.000000,66.000000,50.000000
max,138.000000,543.000000,37.000000,2022.000000,33.000000,106.000000,1.000000,2.000000,330.000000,67.000000,106.000000
